# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [101]:
!pip install langchain chromadb sentence-transformers huggingface-hub -q
!pip install pypdf
!pip install -U langchain-community
!pip install faiss-cpu transformers sentence-transformers

In [43]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: NO


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [102]:
!pip install langchain

In [103]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
!pip install -qU pypdf langchain_community

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [3]:
import pandas as pd

In [10]:
df = pd.read_csv('/content/Dataset.csv')

In [11]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB
None


In [8]:
print(df.describe())

                                                 المخالفة  \
count                                                 104   
unique                                                104   
top     قيادة المركبة في الأسواق التي لا يسمح بالقيادة...   
freq                                                    1   

                               الغرامة  
count                              104  
unique                               7  
top     الغرامة المالية 500 - 900 ريال  
freq                                25  


In [9]:
df.isnull().sum()

,0
المخالفة,0
الغرامة,0


In [13]:
len(df)

104

In [104]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/traffic+violations.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

In [105]:
print(len(docs))

10


In [106]:
print(docs[0].page_content[0:10])
print(docs[0].metadata)

جدول المخا
{'source': '/content/traffic+violations.pdf', 'page': 0}


In [107]:
for i, doc in enumerate(docs):
    print(f"--- Document {i + 1} ---")
    print(doc.page_content)
    print("\n\n")

--- Document 1 ---
جدول المخالفات رقم۱ ) 
الغرامة المالية (150/100ريال ) 
1- قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها 
٢- ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. 
3- عدم وجود تأمين سار للمركبة 
4- عبور المشاة للطرق من غير الأماكن المخصصة لهم 
5- عدم تقيدالمشاة بالإشارات الخاصة بهم.  
6- وقوف المركبة في أماكن غير مخصصة للوقوف 
7- عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم 
  



--- Document 2 ---
جدول المخالفات رقم۲) 
الغرامة المالية (300/150ريال ) 
1- عدم استخدام إشارة الإلتـاف عـنـد التحول لليمين أو اليسار أو التجاوز أو تغيير  المسار. 
٢ - الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين مترا. 
3- قيام سائق الدراجة الآلية أو العادية- أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو 
سحب أو حمل أشياء تعرض مستخدم الطريق للخطر. 
4- مخالفة قواعد استعمال أنوارالتلاقي.  
5- عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة 
6- إساءة استعمال منبه المركبة 
7- عدم تقديم المركبة للفحص الفني الدوري 
8- مخالفة تنظيمات السير على الطرق 
۹- عدم ربط حزام

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [116]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub

In [118]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "API"

In [117]:
embedding_model = HuggingFaceEmbeddings(
    model_name="wissamantoun/araelectra-base-artydiqa",
    model_kwargs={"device": "cuda", "trust_remote_code": True},
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [53]:
embedding_model = HuggingFaceEmbeddings(
    model_name="bhavikardeshna/multilingual-bert-base-cased-arabic",
    model_kwargs={"device": "cuda", "trust_remote_code": True},
)

Some weights of BertModel were not initialized from the model checkpoint at bhavikardeshna/multilingual-bert-base-cased-arabic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [112]:
embedding_model = HuggingFaceEmbeddings(
    model_name="bhavikardeshna/xlm-roberta-base-arabic",
    model_kwargs={"device": "cuda", "trust_remote_code": True},
)

Some weights of XLMRobertaModel were not initialized from the model checkpoint at bhavikardeshna/xlm-roberta-base-arabic and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [119]:
markdown_content = []
for index, row in df.iterrows():
  violation = row['المخالفة']
  fine = row['الغرامة']
  markdown_content.append(f"**{violation}** - {fine}")

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [120]:
docs = [Document(page_content=text) for text in markdown_content]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)

splits = text_splitter.split_documents(docs)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [121]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [122]:
retriever = vectorstore.as_retriever()

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [123]:
from langchain import HuggingFaceHub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature": 0, "max_length": 512}
)


system_prompt = (
    "انت المساعد للاجابة على المهام"
    "استخدم الأجزاء التالية من السياق المسترجع للإجابة "
    "السؤال. إذا كنت لا تعرف الإجابة، قل أنك لا تعرف  "

    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)


rag_chain = create_retrieval_chain(retriever, question_answer_chain)


results = rag_chain.invoke({"input": "ما هي الغرامة على القيادة بدون رخصة؟"})

print(results)


{'input': 'ما هي الغرامة على القيادة بدون رخصة؟', 'context': [Document(metadata={}, page_content='**وضع كتابة أو رسم أو ملصق أو أي بيان آخر على جسم المركبة دون موافقة الجهات المختصة.** - الغرامة'), Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو'), Document(metadata={}, page_content='**دخول الشاحنات والمعدات الثقيلة وما في حكمهما إلى المدن أو الخروج منها في الأوقات غير المسموح'), Document(metadata={}, page_content='**استخدام أجهزة غير مصرح بها في المركبة أو وضع شعارات أو ملصقات تتنافى مع الآداب العامة.** - الغرامة')], 'answer': '                                                                                                                                                                                                                                                               '}


In [124]:
from langchain import HuggingFaceHub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature": 0.3, "max_length": 512}
)


system_prompt = (
    "انت المساعد للاجابة على المهام  "
    "استخدم الأجزاء التالية من السياق المسترجع للإجابة "
    "السؤال. إذا كنت لا تعرف الإجابة، قل أنك لا تعرف  "
    """ستناداً إلى السياق أعلاه، قدم إجابة مفصلة باللغة العربية تتعلق بالمخالفة المرورية المحددة أو المعلومات المطلوبة. يجب أن تتضمن الإجابة التفاصيل المتعلقة بالغرامات المالية أو العقوبات،
    """
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)


rag_chain = create_retrieval_chain(retriever, question_answer_chain)


results = rag_chain.invoke({"input": "ما هي الغرامة عند عدم وجود تامين للمركبة؟"})

print(results)

{'input': 'ما هي الغرامة عند عدم وجود تامين للمركبة؟', 'context': [Document(metadata={}, page_content='**دخول الشاحنات والمعدات الثقيلة وما في حكمهما إلى المدن أو الخروج منها في الأوقات غير المسموح'), Document(metadata={}, page_content='**وضع كتابة أو رسم أو ملصق أو أي بيان آخر على جسم المركبة دون موافقة الجهات المختصة.** - الغرامة'), Document(metadata={}, page_content='**استخدام أجهزة غير مصرح بها في المركبة أو وضع شعارات أو ملصقات تتنافى مع الآداب العامة.** - الغرامة'), Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو')], 'answer': '                                                                                                                                                                                                                                                               '}


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.